# Notes
    1) Check if ns6 data is automatically adjusted by gain and offset
    2) Add 0's to z property in electrodes table
    3) Ask how to set Location information

In [1]:
# import needed toolboxes
from HatLabNWBConverters_with_neuroconv import MarmForageNWBConverter, MarmVideoNWBConverter
from pprint import pprint
from pathlib import Path
from neuroconv.utils import load_dict_from_file
from neuroconv.utils.json_schema import dict_deep_update
from neuroconv.tools.spikeinterface import add_devices, add_electrode_groups
from pynwb import NWBFile, NWBHDF5IO
from pynwb.image import ImageSeries 
from nwbwidgets import nwb2widget
from os.path import join as pjoin
from importlib import reload, sys
import glob
import yaml
import numpy as np
import pandas as pd
import os

sys.path.insert(0, '/project/nicho/projects/marmosets/code_database/data_processing/nwb_tools/hatlab_nwb_tools/')
from hatlab_nwb_functions import *

/project/nicho/environments/spyder/lib/python3.9/site-packages/plotly/graph_objects/__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
/project/nicho/environments/spyder/lib/python3.9/site-packages/plotly/graph_objects/__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
/project/nicho/environments/spyder/lib/python3.9/site-packages/plotly/graph_objects/__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
/project/nicho/environments/spyder/lib/python3.9/site-packages/plotly/graph_objects/__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ip

In [2]:
reload(sys.modules['HatLabNWBConverters_with_neuroconv'])
from HatLabNWBConverters_with_neuroconv import MarmForageNWBConverter, MarmVideoNWBConverter
reload(sys.modules['hatlab_nwb_functions'])
from hatlab_nwb_functions import *

In [3]:
# User defined inputs
dpath = '/project/nicho/data/marmosets/electrophys_data_for_processing/'
app_path = '/project/nicho/data/marmosets/kinematics_videos/test/TYJL/2022_10_24/avi_videos/'
enc_path = '/project/nicho/data/marmosets/kinematics_videos/test_free/TYJL/2022_10_24/avi_videos/'
sess = 'TY20221024_testbattery'
rfile = 'TY20221024_testbattery_001.ns6'
nfile = 'TY20221024_testbattery_001.nev'
mfile = 'all_metadata_test.yml'
sfile = 'tony.yml'
ofile = 'TY20221024_testbattery_001_NWBTEST_only_neural_converters.nwb'
prbfile = '/project/nicho/data/marmosets/prbfiles/TY_array.prb' # name of probe (.prb) file

nwbfile_path = pjoin(dpath, sess, ofile)

filter_dict = {'6': 'None'}
stub_test= False

In [4]:
# Get source_schema
source_schema = MarmVideoNWBConverter.get_source_schema()
source_schema

{'required': [],
 'properties': {'ApparatusBehaviorVideos': {'required': ['file_paths'],
   'properties': {'file_paths': {'type': 'array'}},
   'type': 'object',
   'additionalProperties': False}},
 'type': 'object',
 'additionalProperties': False,
 '$schema': 'http://json-schema.org/draft-07/schema#',
 '$id': 'source.schema.json',
 'title': 'Source data schema',
 'description': 'Schema for the source data, files and directories',
 'version': '0.1.0'}

In [5]:
# Construct the source data
base_path          = pjoin(dpath, sess)
file_recording_raw = pjoin(base_path, rfile)
file_sorting       = pjoin(base_path, nfile)
app_video_files    = sorted(glob.glob(pjoin(app_path, '*s1*.avi')))[:2]
enc_video_files    = sorted(glob.glob(pjoin(enc_path, '*s1*.avi')))[0]

source_data = dict(ApparatusBehaviorVideos=dict(file_paths=app_video_files))

# if len(app_video_files)>0:
#     source_data.update(ApparatusBehaviorVideos=dict(file_paths=app_video_files))    
# if len(enc_video_files)>0:
#     source_data.update(EnclosureBehaviorVideos=dict(file_paths=enc_video_files))  
    
# len(source_data['ApparatusBehaviorVideos']['file_paths'])


In [6]:
# Initialize converter
converter = MarmVideoNWBConverter(source_data=source_data)

print('Data interfaces for this converter:')
pprint(converter.data_interface_objects, width=120)

Source data is valid!
Data interfaces for this converter:
{'ApparatusBehaviorVideos': <neuroconv.datainterfaces.behavior.movie.moviedatainterface.MovieInterface object at 0x7f6a678eabb0>}


In [7]:
# # Get metadata_schema
# metadata_schema = converter.get_metadata_schema()
# #pprint(metadata_schema, width=120)
# metadata_schema['properties']['Behavior']['properties']['Movies']
# # for key in metadata_schema['properties']['Behavior'].keys():
# #     print('%s:' %key)

In [8]:
io = NWBHDF5IO(nwbfile_path, 'r+')
nwbfile = io.read()


In [9]:
nwbmeta = nwbfile.fields
meta_keys = [key for key in nwbmeta.keys() if key in ['session_start_time']]
meta_keys

['session_start_time']

In [10]:
# Get metadata from source data and from file
metadata = converter.get_metadata()

# load exp metadata
#m = load_dict_from_file('/project/nicho/projects/marmosets/code_database/data_processing/nwb_tools/session_metadata.yml')
# m = load_dict_from_file(mfile)
for key in meta_keys:
    metadata = dict_deep_update(metadata, {'NWBFile': {key : nwbmeta[key]}}, append_list=False, remove_repeats=True)

# load subject metadata
# s = load_dict_from_file(sfile)
# metadata = dict_deep_update(metadata, s)
# convert 
converter.validate_metadata(metadata)

Metadata is valid!


In [11]:
# conversion schema
conversion_options_schema = converter.get_conversion_options_schema()
conversion_options = converter.get_conversion_options()
# conversion_options

In [12]:
# build conversion options
conversion_options = converter.get_conversion_options()
conversion_options['ApparatusBehaviorVideos']=dict(stub_test=stub_test, 
                                                   starting_times=[855.66, 900.0],
                                                  )
# get video start times
#conversion_options['KinematicVideos'] = dict(starting_times=kstart_times)

In [99]:
converter.run_conversion(
    metadata=metadata,  
    nwbfile = nwbfile,
    conversion_options=conversion_options,
)
io.write(nwbfile)
io.close()

Metadata is valid!
conversion_options is valid!


retrieving timestamps: 100%|████████████████| 2564/2564 [00:33<00:00, 76.99it/s]


In [100]:
io = NWBHDF5IO(nwbfile_path, 'r')
nwbfile = io.read()
nwb2widget(nwbfile)

ValueError: tmax and tmin were probably entered in the wrong order. tmax should be first

In [101]:
io.close()